In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import OneClassSVM
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

%matplotlib inline
pd.options.mode.chained_assignment = None

In [2]:
def train_test(df):
    ''' on utilise year == 2022 pour constituer le dataset de test'''
    
    train = df.iloc[:30031]
    test =  df.iloc[30032:]
    
    return train, test

In [28]:
def rf_modelisation(x_train, y_train):
    ''' determination des hyperparametre de RF'''
    weights = np.linspace(0.1,0.9,100)
    params = [{
        "n_estimators": [10, 50, 100,150,200,300,500],
        "max_features": [2, 4, 8,10,12,16,20],
        "class_weight":[{0:x, 1:1.0-x} for x in weights]
        }]

    rfCV = GridSearchCV(
        RandomForestClassifier(),
        params,
        scoring="recall",
        cv=5,
        n_jobs=-1,
        return_train_score=True)
    
    rfCV = rfCV.fit(x_train, y_train)
    
    return rfCV.best_estimator_

#
def logist_modelisation(x_train, y_train):
    # grille de valeurs
    weights = np.linspace(0.1,0.9,100)

    params = [{"C": [0.01, 0.1, 0.2, 0.5, 1, 5, 10, 20], 
           'max_iter': [100,250, 500, 700, 900],
           "class_weight":[{0:x, 1:1.0-x} for x in weights]
          }]

    logitCV = GridSearchCV(
        LogisticRegression(),
        params,
        scoring="recall",
        cv=5,
        n_jobs=-1,
        return_train_score=True)
    
    logitCV = logitCV.fit(x_train, y_train)
    
    return logitCV.best_estimator_

#
def OneSVM_modelisation(x_train, y_train):
    # grille de valeurs
    weights = np.linspace(0.1,0.9,100)

    params = [{"nu": [0.1, 0.2, 0.3, 0.5, 0.6, 0.7, 0.9], 
              'max_iter': [100,250, 500, 700, 900],
           #"class_weight":[{0:x, 1:1.0-x} for x in weights]
          }]

    outlierCV = GridSearchCV(
        OneClassSVM(),
        params,
        scoring="recall",
        cv=5,
        n_jobs=-1,
        return_train_score=True)
    
    outlierCV = outlierCV.fit(x_train, y_train)
    
    
    return outlierCV.best_estimator_

In [29]:
# Chargement des données
data = pd.read_csv("data/features.csv", sep=';',low_memory=False)
label = pd.read_csv("data/descriptif.csv", sep=';',low_memory=False)
print(len(data))


# definition de train et test 
# definition de x et y 
x_train, x_test = train_test(data.iloc[:,1:])
train_target,test_target = train_test(label)
y_test = test_target['baignade']
y_train = train_target['baignade']


34561


In [30]:


## Random forest
model_rf = rf_modelisation(x_train, y_train)
model_rf



In [ ]:
## Logistic Regression
model_logist = logist_modelisation(x_train, y_train)
model_logist

c:\Users\FABDELLI\python\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
8400 fits failed out of a total of 16800.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
8400 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\FABDELLI\python\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\FABDELLI\python\lib\site-packages\sklearn\linear_model\_logistic.py", line 1091, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "c:\Users\FABDELLI\python\lib\site-packages\sklearn\linear_model\_logistic.py", line 61, in _check_solver
    raise ValueError

LogisticRegression(C=0.5, class_weight={0: 0.1, 1: 0.9})

In [ ]:


## OneSVM
model_svm = OneSVM_modelisation(x_train, y_train)
model_svm



c:\Users\FABDELLI\python\lib\site-packages\sklearn\model_selection\_search.py:953: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
c:\Users\FABDELLI\python\lib\site-packages\sklearn\model_selection\_search.py:953: UserWarning: One or more of the train scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
c:\Users\FABDELLI\python\lib\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


OneClassSVM(max_iter=100, nu=0.1)

# Evaluation

In [ ]:


## RF
model_rf.fit(x_train, y_train)

# prediction
y_train_predict = model_rf.predict(x_train)
y_test_predict = model_rf.predict(x_test)

# Evaluation
print("-----------------------Training data-----------------------")
print(classification_report(y_train, y_train_predict))
print("-------------------------Test data-------------------------")
print(classification_report(y_test, y_test_predict))



c:\Users\FABDELLI\python\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


-----------------------Training data-----------------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     29638
           1       1.00      0.66      0.80       393

    accuracy                           1.00     30031
   macro avg       1.00      0.83      0.90     30031
weighted avg       1.00      1.00      1.00     30031

-------------------------Test data-------------------------
              precision    recall  f1-score   support

           0       0.88      1.00      0.93      3976
           1       0.00      0.00      0.00       553

    accuracy                           0.88      4529
   macro avg       0.44      0.50      0.47      4529
weighted avg       0.77      0.88      0.82      4529



c:\Users\FABDELLI\python\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\FABDELLI\python\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model_logist.fit(x_train, y_train)

# prediction
y_train_predict = model_logist.predict(x_train)
y_test_predict = model_logist.predict(x_test)

# Evaluation
print("-----------------------Training data-----------------------")
print(classification_report(y_train, y_train_predict))
print("-------------------------Test data-------------------------")
print(classification_report(y_test, y_test_predict))

c:\Users\FABDELLI\python\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


-----------------------Training data-----------------------
              precision    recall  f1-score   support

           0       0.99      0.98      0.99     29638
           1       0.08      0.11      0.10       393

    accuracy                           0.97     30031
   macro avg       0.54      0.55      0.54     30031
weighted avg       0.98      0.97      0.97     30031

-------------------------Test data-------------------------
              precision    recall  f1-score   support

           0       0.89      0.94      0.92      3976
           1       0.31      0.19      0.24       553

    accuracy                           0.85      4529
   macro avg       0.60      0.57      0.58      4529
weighted avg       0.82      0.85      0.83      4529



In [ ]:
model_svm.fit(x_train, y_train)

# prediction
y_train_predict = model_svm.predict(x_train)
y_test_predict = model_svm.predict(x_test)

# Evaluation
print("-----------------------Training data-----------------------")
print(classification_report(y_train, y_train_predict))
print("-------------------------Test data-------------------------")
print(classification_report(y_test, y_test_predict))

c:\Users\FABDELLI\python\lib\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


-----------------------Training data-----------------------
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         0
           0       0.00      0.00      0.00     29638
           1       0.02      0.24      0.03       393

    accuracy                           0.00     30031
   macro avg       0.01      0.08      0.01     30031
weighted avg       0.00      0.00      0.00     30031

-------------------------Test data-------------------------
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         0
           0       0.00      0.00      0.00      3976
           1       0.03      0.06      0.04       553

    accuracy                           0.01      4529
   macro avg       0.01      0.02      0.01      4529
weighted avg       0.00      0.01      0.00      4529



c:\Users\FABDELLI\python\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\FABDELLI\python\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\FABDELLI\python\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\FABDELLI\python\lib\site-packages\sklearn\metrics\_cla

In [ ]:
import joblib
joblib.dump(model_svm, "saved_models/oneclassSVM.joblib")
joblib.dump(model_rf, "saved_models/random_forest.joblib")
joblib.dump(model_logist, "saved_models/reg_logist.joblib")